In [1]:
#写在前面，感谢林大佬的开源数据处理，其次感谢鱼老的lgb参数，我的工作其实就是把他们的代码整合一起了。
#作者- - - - Ambromse
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
#import xgboost as xgb
#import catboost as ctb

In [2]:
#请看林大佬的开源base
path = './ccf_car/'

train_sales_data = pd.read_csv(path + 'train_sales_data.csv')
train_search_data = pd.read_csv(path + 'train_search_data.csv')
train_user_reply_data = pd.read_csv(path + 'train_user_reply_data.csv')

test = pd.read_csv(path + '/evaluation_public.csv')

data = pd.concat([train_sales_data, test], ignore_index=True)
data = data.merge(train_search_data, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user_reply_data, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['forecastVolum']

num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum']
cate_feat = ['bodyType', 'model', 'province']

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [3]:
data.head()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0


In [4]:
num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum']
cate_feat = ['bodyType', 'model', 'province']

for i in cate_feat:
    data[i] = data[i].astype('category')
features = num_feat + cate_feat

In [5]:
data.head()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0


In [6]:
data.nunique()

adcode               22
bodyType              4
id                 5281
model                60
province             22
regMonth             12
regYear               3
popularity         7239
carCommentVolum     571
newsReplyVolum     1159
label              2965
dtype: int64

In [7]:
data.dtypes

adcode                int64
bodyType           category
id                    int32
model              category
province           category
regMonth              int64
regYear               int64
popularity          float64
carCommentVolum     float64
newsReplyVolum      float64
label               float64
dtype: object

In [8]:
print(train_sales_data.shape)
print(data.shape)

(31680, 7)
(36960, 11)


In [9]:
train = data[:31680]
y_label = train['label']
test = data[31680:]

drop_features = ['label']
train = train.drop(drop_features,axis=1)
test = test.drop(drop_features,axis=1)

In [10]:
train.shape
print(type(y_label))
train.head()

<class 'pandas.core.series.Series'>


,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0


In [11]:
#from sklearn.model_selection import KFold,StratifiedKFold

In [12]:
#此处请看鱼老知乎

#开始训练
import time
since = time.time()

x_score = []
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import f1_score
random_seed = 2018
#lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=395, max_depth=-1, learning_rate=0.1, n_estimators=5000, max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,min_child_weight=5, min_child_samples=5, subsample=0.8, subsample_freq=1,colsample_bytree=0.8, reg_alpha=3, reg_lambda=5, seed=1000, n_jobs=-1, silent=True)
lgb_clf = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.01, n_estimators=5000,
                             max_bin=425, subsample_for_bin=50000, objective='regression_l1', min_split_gain=0,min_child_weight=5,
                             min_child_samples=10, subsample=0.8, subsample_freq=1,colsample_bytree=0.8,
                             reg_alpha=3, reg_lambda=0.1, seed=1000, n_jobs=-1, silent=True)

skf = StratifiedKFold(n_splits=5, random_state=random_seed, shuffle=True)
baseloss = []
loss = 0

res_prob = pd.DataFrame()
res = pd.DataFrame()
for i, (train_index, test_index) in enumerate(skf.split(train,y_label)):
    print("Fold", i)
    x_valid,y_valid = train.loc[test_index],y_label.loc[test_index]
    lgb_model = lgb_clf.fit(train.loc[train_index],y_label.loc[train_index],
                            eval_names=['train', 'valid'],
                            eval_metric='mae',                              
                            eval_set=[(train.loc[train_index], y_label.loc[train_index]),   
                                      (train.loc[test_index], y_label.loc[test_index])],early_stopping_rounds=100,verbose = 250)
    x_pred = lgb_model.predict(x_valid, num_iteration=lgb_model.best_iteration_)
    test_pred = lgb_model.predict(test, num_iteration=lgb_model.best_iteration_)
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss / 5)

time_elapsed = time.time() - since
print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) # 打印出来时间

C:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold 0
Training until validation scores don't improve for 100 rounds.
[250]	train's l1: 201.148	train's l1: 201.148	valid's l1: 202.443	valid's l1: 202.443
[500]	train's l1: 160.367	train's l1: 160.367	valid's l1: 165.325	valid's l1: 165.325
[750]	train's l1: 141.234	train's l1: 141.234	valid's l1: 148.735	valid's l1: 148.735
[1000]	train's l1: 127.101	train's l1: 127.101	valid's l1: 136.772	valid's l1: 136.772
[1250]	train's l1: 117.241	train's l1: 117.241	valid's l1: 128.94	valid's l1: 128.94
[1500]	train's l1: 110.036	train's l1: 110.036	valid's l1: 123.266	valid's l1: 123.266
[1750]	train's l1: 104.35	train's l1: 104.35	valid's l1: 118.832	valid's l1: 118.832
[2000]	train's l1: 99.7924	train's l1: 99.7924	valid's l1: 115.323	valid's l1: 115.323
[2250]	train's l1: 95.7769	train's l1: 95.7769	valid's l1: 112.324	valid's l1: 112.324
[2500]	train's l1: 92.3746	train's l1: 92.3746	valid's l1: 109.762	valid's l1: 109.762
[2750]	train's l1: 89.113	train's l1: 89.113	valid's l1: 107.515	va

[1500]	train's l1: 109.972	train's l1: 109.972	valid's l1: 125.236	valid's l1: 125.236
[1750]	train's l1: 103.834	train's l1: 103.834	valid's l1: 120.099	valid's l1: 120.099
[2000]	train's l1: 98.8271	train's l1: 98.8271	valid's l1: 116.105	valid's l1: 116.105
[2250]	train's l1: 94.5268	train's l1: 94.5268	valid's l1: 112.773	valid's l1: 112.773
[2500]	train's l1: 91.1717	train's l1: 91.1717	valid's l1: 110.272	valid's l1: 110.272
[2750]	train's l1: 88.2206	train's l1: 88.2206	valid's l1: 108.021	valid's l1: 108.021
[3000]	train's l1: 85.59	train's l1: 85.59	valid's l1: 106.108	valid's l1: 106.108
[3250]	train's l1: 83.3797	train's l1: 83.3797	valid's l1: 104.531	valid's l1: 104.531
[3500]	train's l1: 81.3691	train's l1: 81.3691	valid's l1: 103.202	valid's l1: 103.202
[3750]	train's l1: 79.6179	train's l1: 79.6179	valid's l1: 102.154	valid's l1: 102.154
[4000]	train's l1: 78.0057	train's l1: 78.0057	valid's l1: 101.088	valid's l1: 101.088
[4250]	train's l1: 76.6253	train's l1: 76.6253	

In [18]:
best=pd.read_csv('./rst/stacking_lgb_0_46And0_49_rst_0.507.csv')
testsub = pd.read_csv(path + '/evaluation_public.csv')
testsub.head()
sum_proba = res.sum(axis=1) / 5.0
testsub['forecastVolum'] = sum_proba
testsub['forecastVolum'] =testsub['forecastVolum'].astype(int)
testsub['forecastVolum'][testsub['forecastVolum']<0] =  0
#testsub['forecastVolum']= (testsub[['forecastVolum']]*0.4+0.6*best['forecastVolum'])
testsub[['id','forecastVolum']].to_csv('3709_best_jiaquan.csv',header=True,index=None)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
